In [0]:
import tensorflow as tf
import numpy as np
import os

## Data Preparation

In [0]:
os.chdir("/content/drive/My Drive/IDL-Ex-Colab/resources/ass6")

In [0]:
!python prepare_data2.py shakespeare_input.txt skp \\n\\n+ -m 500

In [57]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'N': 3, ']': 4, '!': 5, 'w': 6, 'l': 7, 'e': 8, 'M': 9, 'u': 10, '$': 11, ';': 12, 'o': 13, 'p': 14, 'I': 15, 'c': 16, 'D': 17, 'P': 18, 'F': 19, '.': 20, 'H': 21, 'T': 22, 'U': 23, ':': 24, 'a': 25, 'j': 26, 'h': 27, 'z': 28, 'B': 29, 'v': 30, 'g': 31, '-': 32, 'x': 33, 'i': 34, "'": 35, 'r': 36, 'y': 37, '&': 38, ',': 39, 'd': 40, 'b': 41, 'X': 42, 'k': 43, 'R': 44, 't': 45, 'W': 46, 'A': 47, 's': 48, 'f': 49, 'L': 50, 'O': 51, 'J': 52, 'Q': 53, ' ': 54, 'G': 55, 'Z': 56, '3': 57, 'm': 58, 'C': 59, 'q': 60, 'K': 61, 'Y': 62, 'V': 63, '[': 64, 'n': 65, 'S': 66, 'E': 67, '?': 68, '\n': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}
70


### Making fixed size sequence. 

In [0]:
dataset = data.padded_batch(128, padded_shapes=([499]) , drop_remainder=True).shuffle(100000)

## Model architecture

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None], mask_zero=True),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

## Model execution

In [77]:
# Parameters
epochs = 40
learning_rate = 0.001
batch_size = 128
embedd_size = 256
rnn_units = 1024

# Making Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Building Model
model = build_model(vocab_size, embedd_size, rnn_units, batch_size)

for epoch in range(epochs):
  losses = []

  ## Handling padding manually
  # dataset = data.padded_batch(batch_size, padded_shapes=([500]) , drop_remainder=True).shuffle(100000)

  ## Delegating handling of padding to Model 
  dataset = data.padded_batch(batch_size, drop_remainder=True).shuffle(100000)

  for x_batch in dataset:
    input = x_batch
    target = x_batch[:, 1:]
    mask_val = tf.math.count_nonzero(tf.sequence_mask(input, 500)).numpy()
    model.reset_states()

    with tf.GradientTape() as tape:
      predictions = model(input)
      loss = tf.reduce_sum(
          tf.keras.losses.sparse_categorical_crossentropy(
              target, predictions[:, :-1], from_logits=True))
      ## Calculating mean loss
      loss = loss/mask_val
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    losses.append(loss)
    # End of inner for-loop
  
  print("Epoch: {}, Loss: {}".format(epoch, np.array(losses).mean()))
  # End of outter for-loop

# Saving the model
model.save('model/model1')

Epoch: 0, Loss: 0.4641973376274109
Epoch: 1, Loss: 0.4032205641269684
Epoch: 2, Loss: 0.3545578122138977
Epoch: 3, Loss: 0.30976733565330505
Epoch: 4, Loss: 0.2686918377876282
Epoch: 5, Loss: 0.23100578784942627
Epoch: 6, Loss: 0.19692136347293854
Epoch: 7, Loss: 0.166824072599411
Epoch: 8, Loss: 0.1410035640001297
Epoch: 9, Loss: 0.1195097491145134
Epoch: 10, Loss: 0.1020113155245781
Epoch: 11, Loss: 0.08803224563598633
Epoch: 12, Loss: 0.07688817381858826
Epoch: 13, Loss: 0.06801409274339676
Epoch: 14, Loss: 0.060951586812734604
Epoch: 15, Loss: 0.0552423894405365
Epoch: 16, Loss: 0.050562672317028046
Epoch: 17, Loss: 0.046557217836380005
Epoch: 18, Loss: 0.043306730687618256
Epoch: 19, Loss: 0.04048481583595276
Epoch: 20, Loss: 0.0381179116666317
Epoch: 21, Loss: 0.03601647540926933
Epoch: 22, Loss: 0.03425099328160286
Epoch: 23, Loss: 0.0327029675245285
Epoch: 24, Loss: 0.03135411813855171
Epoch: 25, Loss: 0.03012748807668686
Epoch: 26, Loss: 0.02911739982664585
Epoch: 27, Loss: 0.

In [106]:
model = tf.keras.models.load_model('model/model1')

## Text generation

In [182]:
# Making model compatibile for Batch size 1
prediction_model = build_model(vocab_size, embedd_size, rnn_units, 1)
prediction_model.set_weights(model.get_weights())

# First character
start_string = 'k'
start_indices = vocab[start_string]
start_indices = tf.expand_dims([start_indices], axis=0)

# Resetting model state
prediction_model.reset_states()
softmax_list = []
vocab_list = list(range(vocab_size))

## Generation
for time_step in range(1000):
  logits = prediction_model(start_indices)
  logits = tf.squeeze(logits, 0)
  out_t = tf.nn.softmax(logits, 1)
  index = np.random.choice(vocab_list , p = out_t.numpy().flatten())
  softmax_list.append(index)

  # GLITCH: Model breaks at index 0 <PAD> value. 
  if index == 0:
    index = np.random.randint(1, vocab_size)
  
  start_indices = tf.expand_dims([index], axis=0)


# print(softmax_list)
seq = [ind_to_ch[ind] for ind in softmax_list]
print("".join(seq))

en: repair to
Hell, master, that have taken your wife with
us, if you find it, welcome: if you think of no
blewhing, I'll give thee thy face, and it husbands.
White course, my lord! you know I what you would.</S><PAD>JAMIA:</S><PAD></S><PAD>BASSANIO:
Thou didst prove possess'd as soon stay'd before,
So idly too much for the nobles?</S><PAD></S><PAD>PHERO:
If it please you to say, it due so.</S><PAD>
As thou art to thy person;
For here's a sister of the stede, sir,
Are mighty odds being appointed.</S><PAD>ones: I
am almost as well as She shall be Richard.</S><PAD>OSERICHARDES:
I'll have a sickness of my chin. Give me
Letters from this motive.</S><PAD> stagger me.</S><PAD>llow.</S><PAD>nder: distrust
No watching ever firm and not broke.</S><PAD>OTH
Let us withdraw.</S><PAD>aly in pride.</S><PAD>IAGO:
I have been told so, master.</S><PAD> AENensitors,
Why sinks my peace what would make haste.</S><PAD> ANDenten
Your mother chides, welcome hither.</S><PAD> ADWALD:
'Tis already,
And make us 